Using the Google API to find different store types in Davidson County by District

This code is modified from https://github.com/raom1/nashville-food-deserts/blob/master/test_notebooks/grocery_google_api_test.ipynb

In [1]:
import requests
import json
import time
import pickle
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon
# import censusdata
import pandas as pd
import math
from sklearn.metrics.pairwise import haversine_distances
from sklearn.cluster import AgglomerativeClustering
from bs4 import BeautifulSoup as BS
import ast
from itertools import combinations
from collections import Counter
#from scipy.cluster.hierarchy 

%matplotlib inline

Import API Key from a secure file

In [2]:
hidden = pd.read_pickle('../data/API_key.pkl')
api_key = hidden.loc[0][1]

In [12]:
location = '36.186314,-87.0654304' # lat, lng as a string
radius = '322000' # in meters, 20 miles, approximate distance from Davidson center to furthest edge
types = 'grocery_or_supermarket' # select single store type as a test case

In [4]:
all_types = ['bakery',
             'convenience_store',
             'department_store',
             'drugstore',
             'gas_station',
             'grocery_or_supermarket',
             'home_goods_store',
             'supermarket',
             'pharmacy']

In [13]:
# Modified from https://python.gotrained.com/google-places-api-extracting-location-data-reviews/

def search_places_by_coordinate(location, radius, types, api_key, sleep_sec = 2):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places = []
    params = {
        'location': location,
        'radius': radius,
        'types': types,
        'key': api_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  res.json()
    places.extend(results['results'])
    time.sleep(sleep_sec)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = res.json()
        places.extend(results['results'])
        time.sleep(sleep_sec)
    return places

In [14]:
gas_station_20_mi_radius = search_places_by_coordinate(location, radius, types, api_key)

In [15]:
len(gas_station_20_mi_radius)

60

In [8]:
print(gas_station_20_mi_radius)

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 36.16341249999999, 'lng': -86.779421}, 'viewport': {'northeast': {'lat': 36.1647340802915, 'lng': -86.77817476970849}, 'southwest': {'lat': 36.1620361197085, 'lng': -86.78087273029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png', 'icon_background_color': '#4B96F3', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/shoppingcart_pinlet', 'name': 'Savi Urban Market', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2269, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116697713471305132430">Savi Urban Market</a>'], 'photo_reference': 'Aap_uEABeVaLzPxGzhyzTHv6xZlcYDaJReOGjMOdCHAZGSKhAnT3yPF6m-m2K-vRhCP_c5NU87g7ZQNoEqv4cvcUAJrTOFZVpcw-NgjjreeHCDhn0L0qvur1rGmVg6PXr9w0tQLXa8RHKwA_Mf_9GQ7yFw4FN-pM7-gOovPsssToc6eUwJOv', 'width': 4032}], 'place_id': 'ChIJPStOVVhmZIgRso0FW3wu3Hw', 'plus_code': {'compound_code': '567C+96 Nashvill

In [9]:
nash_map = folium.Map([36.1612, -86.7775])
for store in gas_station_20_mi_radius:
    coords = store['geometry']['location']
    lat = coords['lat']
    lng = coords['lng']
    folium.Marker([lat, lng]).add_to(nash_map)
nash_map

In [10]:
multiple_stores_20_mi_radius = search_places_by_coordinate(location, radius, all_types, api_key)

In [11]:
len(multiple_stores_20_mi_radius)

21